First, upload your audio file with some speech

In [1]:
from IPython.display import Audio
import numpy as np
import librosa
import torch
from scipy.io import wavfile
import soundfile as sf

In [2]:
audio_filename = '/content/Welcome.wav'
Audio(audio_filename)

In [4]:
data = wavfile.read(audio_filename)
frame_rate = data[0] #fps
speech = data[1] #number of data points (frames in this case)
print('Frame Rate (fps): ', frame_rate, 'Hz')
print('Audio Duration (s): ', len(speech)/frame_rate, 's')

Frame Rate (fps):  16000 Hz
Audio Duration (s):  40.352 s


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: WavFileWarning: Chunk (non-data) not understood, skipping it.
  """Entry point for launching an IPython kernel.


Speech to Text using Huggingface and Wav2Vec-2

In [5]:
!pip install transformers

     |████████████████████████████████| 1.8MB 5.7MB/s 
     |████████████████████████████████| 890kB 12.4MB/s 
     |████████████████████████████████| 3.2MB 10.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=0145d130ec49f82501334e10bb7582e3026c88a5d8b1007c3b7a1eee078b112b
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [6]:
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")

In [15]:
in_audio, _ = librosa.load(audio_filename, sr = 16000)
input_ = tokenizer(in_audio, return_tensors='pt').input_values

In [16]:
logits = model(input_)
logits

CausalLMOutput([('logits',
                 tensor([[[ 15.4335, -27.5253, -27.1835,  ...,  -5.7467,  -7.2729,  -8.1051],
                          [ 15.4085, -27.6929, -27.3558,  ...,  -5.8652,  -7.3127,  -8.1350],
                          [ 15.3838, -27.6636, -27.3222,  ...,  -5.8122,  -7.3205,  -8.1314],
                          ...,
                          [ 15.4812, -27.7237, -27.3809,  ...,  -5.8706,  -7.1700,  -8.0742],
                          [ 15.6129, -27.3852, -27.0234,  ...,  -5.6476,  -7.2012,  -8.1333],
                          [ 15.5788, -27.5797, -27.2221,  ...,  -5.8697,  -7.3818,  -8.2484]]],
                        grad_fn=<AddBackward0>))])

In [18]:
logits.logits.shape

torch.Size([1, 2017, 32])

In [22]:
final_logits = logits.logits
predictions = torch.argmax(final_logits, dim = -1)
len(predictions[0])

2017

In [29]:
output_text = tokenizer.batch_decode(predictions)
output_text[0]

'THANK YOU FOR CHOUSING THE OLYMPUS DICTATION MANAGEMENT SYSTEM THE OLYMPUS DICTATION MANAGEMENT SYSTEM GIVES YOU THE POWER TO MANAGE YOUR DICTATIONS TRANSCRIPTIONS AND DOCUMENTS SEEMLESSLY AND TO IMPROVE THE PRODUCTIVITY OF YOUR DAILY WORK FOR EXAMPLE YOU CAN AUTOMATICALLY SENT THE DICTATION FILES OR TRANSCRIBED DOCUMENTS TO YOUR ASSISTANT OF THE AUTHOR VIRE EMALE OR F T P IF YOURE USING THE SPEECH RECOGNITION SOFTWARE THE SPEECH RECOGNITION ENGINE WORKS IN THE BACKGROUND TO SUPPORT YOUR DOCUMENT CREATION WE HOPE YOU ENJOY THE SIMPLE FLEXIBLE RELIABLE AND SECURE SOLUTIONS FROM OLYMPUS'